# Dataset Preparation for Bakery dataset for Meta-Learning

> Some pre-processings steps implemented to prepare the bakery dataset for meta-learning

In [ ]:
#| default_exp datasets.bakery

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import logging
logging.basicConfig(level=logging.INFO)

import numpy as np
import os
import pandas as pd

In [ ]:
#| export

class BakeryDatasetLoader():

    """ Class to download the Kaggle M5 dataset and apply some preprocessing steps
    to prepare it for application in inventory management. """

    def __init__(self, data_path, overwrite=False, product_as_feature=False, store_as_features=False):
        self.create_paths(data_path)
        self.store_as_features = store_as_features
        self.product_as_feature = product_as_feature
    
    def load_dataset(self):

        """ Main function to load the dataset. """

        logging.info("Importing data")
        self.import_from_folder()
        logging.info("Preprocessing data")
        self.preprocess_pipeline()

        return self.demand, self.SKU_features, self.time_features, self.time_SKU_features, self.mask

    def create_paths(self, data_path):

        """ Create the paths for the data files. """

        self.data_path = data_path
        self.features_path = os.path.join(data_path, "bakery_data.csv")
        self.demand_path = os.path.join(data_path, "bakery_target.csv")

    def preprocess_pipeline(self):
    
        """ Apply simple preprocessing steps to the data. """
        
        logging.info("--Creating catogory mapping and features")  
        # self.sale["id"] = self.sale["id"].str.replace("_evaluation", "")
        unique_mapping = self.features[['store', 'product']].drop_duplicates()
        unique_mapping["store"] = unique_mapping["store"].astype(str)
        unique_mapping["product"] = unique_mapping["product"].astype(str)
        unique_mapping["SKU_id"] = unique_mapping["product"] + "_" + unique_mapping["store"]
        unique_mapping.set_index("SKU_id", inplace=True)

        dummy_columns = []
        if self.product_as_feature:
            dummy_columns.append("product")
        if self.store_as_features:
            dummy_columns.append("store")
        if dummy_columns:
            categories = pd.get_dummies(unique_mapping[dummy_columns], drop_first=True)

        # logging.info("--Preparing sales time series data")
        # id = self.sale["id"]
        # self.sale = self.sale.iloc[:,6:]
        # self.sale["SKU_id"] = id
        # self.sale.set_index("SKU_id", inplace=True)
        # self.sale = self.sale.transpose()
        # self.sale.reset_index(inplace=True, drop=True)
        # self.sale.rename_axis(None, axis=1, inplace=True)
        # self.sale = self.sale[unique_mapping.index]
        
        logging.info("--Preparing calendric information")

        
        calendar_data = self.features[["date", "weekday", "month", "year"]]
        date_as_time = pd.to_datetime(calendar_data['date']).copy().unique()

        # get only rows with unique dates
        calendar_data = calendar_data.drop_duplicates()

        calendar_data['date'] = date_as_time

        # Adding the 'trend' column which counts from 0 and increments by 1
        calendar_data['trend'] = range(len(calendar_data))

        # Adding the 'quarter' column
        calendar_data['quarter'] = calendar_data['date'].dt.quarter

        # Defining the seasonal cut-off dates
        def get_season(date):
            year = date.year
            # Defining specific dates for season changes
            winter_start = pd.Timestamp(year=year, month=12, day=21)
            spring_start = pd.Timestamp(year=year, month=3, day=20)
            summer_start = pd.Timestamp(year=year, month=6, day=21)
            autumn_start = pd.Timestamp(year=year, month=9, day=22)
            
            if date >= winter_start or date < spring_start:
                return "Winter"
            elif spring_start <= date < summer_start:
                return "Spring"
            elif summer_start <= date < autumn_start:
                return "Summer"
            else:
                return "Autumn"

        # Adding the 'season' column with updated season logic
        calendar_data['season'] = calendar_data['date'].apply(get_season)

        dummy_columns = ["weekday", "month", "quarter", "season"]
        self.calendar = pd.get_dummies(calendar_data, columns=dummy_columns, drop_first=True)

        logging.info("--Preparing state-specific features")
        # snap_features = self.calendar[["snap_CA", "snap_TX", "snap_WI"]].copy()
        # self.calendar.drop(["snap_CA", "snap_TX", "snap_WI"], axis=1, inplace=True)

        # new_snap_features = pd.DataFrame(index=range(snap_features.shape[0]), columns=unique_mapping.index)
        # for sku in unique_mapping.index:
        #     state_code = sku.split('_')[-2]
        #     snap_column = f"snap_{state_code}"
        #     new_snap_features[sku] = snap_features[snap_column].values
        # snap_features = new_snap_features

        # logging.info("--Preparing price information")
        # self.price["SKU_id"] = self.price["item_id"] + "_" + self.price["store_id"]
        # self.price.drop(["store_id", "item_id"], axis=1, inplace=True)
        # self.price = self.price.pivot_table(index='wm_yr_wk', columns=['SKU_id'], values='sell_price')
        # self.price = self.price[unique_mapping.index]

        # logging.info("--Creating indicator table if products are available for purchase")
        # self.available = self.price.copy()
        # self.available = self.available.notnull().astype(int)
        # # fill missing values for price (indicated in the available table)
        # self.price.fillna(0, inplace=True)

        # logging.info("--Preparing final outputs and ensure consistency of time and feature dimensions")
        # wm_yr_wk_per_day = self.calendar[["wm_yr_wk"]]
        # self.price = self.price.reset_index()
        # missing_wm_yr_wk_in_price = wm_yr_wk_per_day[~wm_yr_wk_per_day["wm_yr_wk"].isin(self.price["wm_yr_wk"])]
        # if not missing_wm_yr_wk_in_price.empty:
        #     raise ValueError("The following wm_yr_wk values are in calendar but not in price: ", missing_wm_yr_wk_in_price.tolist())
        # self.price = self.price.merge(wm_yr_wk_per_day, on="wm_yr_wk", how="right")
        # self.price.drop(["wm_yr_wk"], axis=1, inplace=True)
        # self.available = self.available.reset_index()
        # self.available = self.available.merge(wm_yr_wk_per_day, on="wm_yr_wk", how="right")
        # self.available.drop(["wm_yr_wk"], axis=1, inplace=True)
        # self.calendar.drop(["wm_yr_wk"], axis=1, inplace=True)

        # price_multi_index = pd.MultiIndex.from_product([['Price'], self.price.columns], names=['Type', 'SKU'])
        # self.price.columns = price_multi_index
        # snap_multi_index = pd.MultiIndex.from_product([['Snap'], snap_features.columns], names=['DataType', 'SKU'])
        # snap_features.columns = snap_multi_index
        # time_SKU_features = pd.concat([self.price, snap_features], axis=1)
       
        # self.demand = self.sale
        # self.SKU_features = categories # features that are not time-dependent
        # self.time_features = self.calendar # features that are time-dependent
        # self.time_SKU_features = time_SKU_features # features taht are time- and SKU-dependent
        # self.mask = self.available # A mask that can either mask datapoints during training or be used as a feature


               
        self.demand = None
        self.SKU_features = None # features that are not time-dependent
        self.time_features = None # features that are time-dependent
        self.time_SKU_features = None # features taht are time- and SKU-dependent
        self.mask = None # A mask that can either mask datapoints during training or be used as a feature

    def import_from_folder(self):
        
        """ Import data from a folder. """

        self.features = pd.read_csv(self.features_path)
        self.demand = pd.read_csv(self.demand_path)

In [ ]:
run_test = True
if run_test:
    data_path = "/Users/magnus/Documents/02_PhD/03_Newsvendor_foundation_model/experiments/datasets/raw/bakery" # For testing purposes, please specify the path to the data on your machine
    if data_path is not None:
        loader = BakeryDatasetLoader(data_path, overwrite=False, product_as_feature=False)
        demand, SKU_features, time_features, time_SKU_features, mask = loader.load_dataset()

INFO:root:Importing data
INFO:root:Preprocessing data
INFO:root:--Creating catogory mapping and features
INFO:root:--Preparing calendric information
INFO:root:--Preparing state-specific features


           date weekday month  year
0    2016-01-02     FRI   JAN  2016
1    2016-01-03     SAT   JAN  2016
2    2016-01-04     SUN   JAN  2016
3    2016-01-05     MON   JAN  2016
4    2016-01-06     TUE   JAN  2016
...         ...     ...   ...   ...
1210 2019-04-26     THU   APR  2019
1211 2019-04-27     FRI   APR  2019
1212 2019-04-28     SAT   APR  2019
1213 2019-04-29     SUN   APR  2019
1214 2019-04-30     MON   APR  2019

[1215 rows x 4 columns]
           date weekday month  year  trend
0    2016-01-02     FRI   JAN  2016      0
1    2016-01-03     SAT   JAN  2016      1
2    2016-01-04     SUN   JAN  2016      2
3    2016-01-05     MON   JAN  2016      3
4    2016-01-06     TUE   JAN  2016      4
...         ...     ...   ...   ...    ...
1210 2019-04-26     THU   APR  2019   1210
1211 2019-04-27     FRI   APR  2019   1211
1212 2019-04-28     SAT   APR  2019   1212
1213 2019-04-29     SUN   APR  2019   1213
1214 2019-04-30     MON   APR  2019   1214

[1215 rows x 5 columns]


In [ ]:
demand

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()